In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import joblib
import torch
import tqdm
import numpy as np
import seaborn as sns


from data import KelpNCDataset
import shared
import torch_deeplabv3 as dlv3

In [ ]:
ds = KelpNCDataset(img_nc_path="data_ncf/train_imgs_fe.nc", mask_nc_path="data_ncf/train_masks.ncf")
_, _, ds_test = dlv3.get_dataset(use_channels=None, random_seed=shared.GLOBAL_SEED)

In [ ]:
y_test = ds_test.masks
y_test

In [ ]:
_, h, w = y_test.shape
y_test_kf = y_test.sum(dim=("i", "j")) / (h * w)
y_test_kf = y_test_kf.compute()
y_test_kf

In [ ]:
np.argsort(-y_test_kf)

In [ ]:
scores, y_hat_aa, _  = joblib.load("ens_dlv3/dev/pred_dlv3_test.joblib")
y_hat_aa.shape

In [ ]:
y_hat_aa_mean = torch.mean(y_hat_aa.float(), dim=0)
y_hat_aa_mean.shape

In [ ]:
plt.contour(y_hat_aa_mean[50], origin="lower", levels=[1])

In [ ]:
plt.imshow(y_hat_aa_mean[49], origin="lower")

In [ ]:
fig, ax = plt.subplots()
i = 41
ax.contour(y_test[i], levels=[0])  # true
ax.imshow(y_hat_aa_mean[i], origin="lower", cmap="pink_r")  # pred

In [ ]:
y_hat_clf = joblib.load("pred_clf_test_agg_aa.joblib")
y_hat_clf.shape

In [ ]:
fig, ax = plt.subplots()
i = 41
ax.contour(y_test[i], levels=[0])  # true
ax.imshow(y_hat_clf[i], origin="lower", cmap="pink_r")  # pred

In [ ]:
# Read leaderboard and set column title to "score" since data does not have a header
lb_me = 38 - 1
lb = pd.read_csv("leaderboard.txt")
lb

In [ ]:
fig, ax = plt.subplots(figsize=(5, 8))
lb_ticks = np.array([1, 5, 10, 25, lb_me + 1, 50, 100]) - 1
ax.barh(-lb.index, lb["score"], height=1, color="lightgrey")
ax.barh(-lb_me, lb["score"][lb_me], color="red", height=1)
ax.set_xlabel("Dice score")
ax.set_yticks(-lb_ticks)
ax.set_yticklabels(lb_ticks + 1)
ax.set_ylabel("Rank")
ax.margins(0)